In [ ]:
!pip install sentence_transformers

In [7]:
!pip install pinecone-client

'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [16]:
!pip install tqdm

'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [17]:
from sentence_transformers import SentenceTransformer
import os
import json
import ast
from tqdm import tqdm
import time

In [9]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)
pinecone_keys = api_keys['pinecone_key1']

In [3]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Anaconda3\envs\PTLLM\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\정수채\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
print(model.encode('Hello?'))

[-4.81911525e-02  5.16481698e-02  6.03663363e-02  7.82035068e-02
 -7.17001185e-02 -7.75212869e-02  1.06707036e-01  1.68510713e-02
 -1.04289174e-01 -5.23629598e-02  1.65249016e-02  8.49034463e-04
  6.14056410e-03 -3.37603614e-02  8.00148174e-02 -9.73476470e-03
 -5.94151486e-03 -5.50033189e-02 -1.58078060e-01  3.91182713e-02
 -7.94542804e-02  3.65013480e-02 -9.79444012e-03  2.21124217e-02
 -2.56253369e-02 -7.71860629e-02  3.74883451e-02  6.86489791e-02
 -7.04608019e-03 -6.37769103e-02  5.05503034e-03  4.70280424e-02
  7.27864057e-02  5.82603039e-03 -1.14917383e-02  7.95496479e-02
 -9.11241025e-02 -6.52544945e-02  5.51138520e-02  1.01117045e-03
 -1.00672916e-02 -2.67531332e-02 -1.24660917e-02 -4.25061248e-02
  3.75460982e-02 -5.03433421e-02  6.59254985e-03  4.99172397e-02
  7.97265619e-02 -2.81720422e-02 -6.57626539e-02 -9.47098583e-02
 -2.92463619e-02  1.93909537e-02  1.14195898e-01 -7.46206660e-03
 -1.19604813e-02 -2.93354485e-02  2.28024423e-02 -2.02381592e-02
 -1.45187061e-02  5.79499

In [18]:
with open('data/documents.txt', 'r', encoding="UTF-8") as f:
    lines = f.readlines()

embedding_list = []
for line in tqdm(lines):
    response = model.encode(line)    
    embedding_list.append(response)  

100%|██████████| 41754/41754 [08:04<00:00, 86.11it/s]


In [19]:
# 백업
with open('data/miniLM-embedded-vector', 'w', encoding='utf-8') as f:
    f.write(str(embedding_list))

In [ ]:
# 로드
with open('data/miniLM-embedded-vector', 'r', encoding='utf-8') as f:
    embedded_vector = f.read()
    embedding_list = ast.literal_eval(embedded_vector)

print(embedding_list)

In [13]:
em_vectors = []
for i, em in enumerate(embedding_list):
    em_vectors.append({"id": str(i), "values": em, "metadata": {"assertion": lines[i]}})
print(len(em_vectors))

5


In [14]:
from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_keys)

index = pc.Index("minilm")

index.upsert(
    vectors=em_vectors
)

{'upserted_count': 5}